In [215]:
import pandas as pd

class Datamodel:
    def __init__(self,supplier_data_path,mmx_path,mnp_path,product_name,product_id,COUNTRY_NAME,destination_lookup_path,customeraccount,latency_data_path,delivery_data_path,Country_Region_Mapping_path,customer_lookup_path,customer_pricelist_path):
        self.supplier_data_path = supplier_data_path
        self.mmx_path = mmx_path
        self.mnp_path = mnp_path
        self.product_name = product_name
        self.product_id = product_id
        self.COUNTRY_NAME = COUNTRY_NAME
        self.destination_lookup_path = destination_lookup_path
        self.customeraccount = customeraccount
        self.latency_data_path = latency_data_path
        self.delivery_data_path = delivery_data_path
        self.Country_Region_Mapping_path = Country_Region_Mapping_path
        self.customer_lookup_path = customer_lookup_path
        self.customer_pricelist_path = customer_pricelist_path
    def supplier(self):
        supplier = pd.read_csv(self.supplier_data_path)
        supplier = supplier.loc[supplier['COUNTRY_NM']==self.COUNTRY_NAME]
        supplier = supplier.loc[supplier['PROD_COVERAGE']=="Y"]
        supplier.drop('AVAILABLE_IN_ROUTING',axis=1,inplace=True)
        supplier = supplier[['PROD_ID','SUPPLIER_ACC_ID','SUPP_ACC_INST','SUPP_SMPP_MAX_CONN','COUNTRY_NM','PROD_NM','SUPPLIER_ACC_NM','NETWORK_NM','CONNECTIVITY', 'DEST_VPC','MCC', 'MNC', 'SUPP_DLR_SUPP', 'CHARGING_MODEL', 'SUPP_TRFF_THROTT', 'MT_COST', 'COVERAGE_FL', 'CONCAT_SUPPORT', 'ROUTE_TYPE', 'MNP_SUPPORT', 'DATACODE_SUPPORT', 'DR_HANDSET', 'DR_SMSC']]
        supplier.drop_duplicates(inplace=True)
        supplier.reset_index(drop=True, inplace=True)
        return supplier
    def adding_tps(self):
        supplier=self.supplier()
        supp = supplier[['PROD_ID','SUPPLIER_ACC_ID','COUNTRY_NM','PROD_NM','SUPPLIER_ACC_NM','NETWORK_NM','CONNECTIVITY', 'DEST_VPC','MCC', 'MNC', 'SUPP_DLR_SUPP', 'CHARGING_MODEL', 'MT_COST', 'COVERAGE_FL', 'CONCAT_SUPPORT', 'ROUTE_TYPE', 'MNP_SUPPORT', 'DATACODE_SUPPORT', 'DR_HANDSET', 'DR_SMSC']]
        supp1=supplier[['SUPP_ACC_INST','SUPPLIER_ACC_ID','SUPP_SMPP_MAX_CONN','SUPP_TRFF_THROTT','DEST_VPC']]
        supp1['TPS']= (supp1['SUPP_TRFF_THROTT'].astype(int)*supp1['SUPP_SMPP_MAX_CONN'].astype(int))
        so=supp1[['SUPP_ACC_INST','SUPPLIER_ACC_ID','DEST_VPC','TPS']]
        so.drop_duplicates(inplace=True)
        so= so.groupby(['SUPPLIER_ACC_ID','DEST_VPC']).agg({'TPS':'sum'})
        so=so.reset_index(level=[0,1])
        so2 = supp.merge(so,left_on=["SUPPLIER_ACC_ID","DEST_VPC"],right_on=["SUPPLIER_ACC_ID","DEST_VPC"], how='left', indicator=True)
        so2.drop_duplicates(inplace=True)
        so2.drop('_merge',axis=1,inplace=True)
        return so2
    def mnp_cost(self): 
        so2=self.adding_tps()
        mmx=pd.read_csv(self.mmx_path)
        mmx1=mmx.loc[(mmx['Product Name']==self.product_name) & (mmx['Country']==self.COUNTRY_NAME)]
        if (mmx1['MNP Enabled']=='NO').all():
            so2['MNP COST']=0
        elif (mmx1['MNP Enabled']=='YES').all():
            mnp=pd.read_excel(self.mnp_path)
            mnp['Countries']=mnp['Countries'].str.upper()
            mnp=mnp.loc[(mnp['Countries']==self.COUNTRY_NAME) & (mnp['Active']== 1)]
            Price=list(mnp['Price'])
            so2['MNP COST']=Price[0]
        else:
            so2['MNP COST']="NA"
        so2['COST_PER_SMS'] = pd.to_numeric(so2['MT_COST'])+ pd.to_numeric(so2['MNP COST'])
        indexCost = so2[(so2['COST_PER_SMS'] == 0)].index
        so2.drop(indexCost , inplace=True)
        return so2
    def destination_of_country(self):
        dest_lookup=pd.read_csv(self.destination_lookup_path)
        country_name = self.COUNTRY_NAME
        dest_lookup=dest_lookup.loc[dest_lookup['country_nm']==country_name]
        vpc=dest_lookup['vpc'].to_list()
        return vpc
    def latency(self):
        latency=pd.read_csv(self.latency_data_path)
        vpc = self.destination_of_country()
        latency['requesttime'] = pd.to_datetime(latency['requesttime'],dayfirst=True)
        latency = latency.sort_values(by=['requesttime']).reset_index(drop=True)
        latency = latency.loc[(latency['customeraccount']==self.customeraccount)& (latency['destinationid'].isin(vpc))].reset_index(drop=True)
        latency['requesttime'] = pd.to_datetime(latency['requesttime'],dayfirst=True)
        latency['requesttime'] = latency['requesttime'].dt.strftime('%d-%m-%Y %H:%M')
        latency['requesttime'] = pd.to_datetime(latency['requesttime'], format='%d-%m-%Y %H:%M')
        latency = latency.loc[latency['servicetype']=='mt'].reset_index(drop=True)
        latency = latency[['requesttime','customeraccount' ,'supplierid','destinationid','productid','servicetype','e2elatencycount','e2elatencysum','acklatencycount','acklatencysum','platform_0_250ms_count','platform_250_500ms_count','platform_500_1000ms_count','platform_1_3s_count','platform_3_5s_count','platform_5_10s_count','platform_over10s_count','platformlatencycount','platformlatencysum','delivery_0_5s_count','delivery_5_10s_count','delivery_10_15s_count','delivery_15_30s_count','delivery_30_60s_count','delivery_60_120s_count','delivery_120_180s_count','delivery_3_10m_count','delivery_10_30m_count','delivery_30_360m_count','delivery_6_24h_count','delivery_over24h_count','deliverylatencycount','deliverylatencysum']]
        latency['platform_0_500ms_count']=latency['platform_0_250ms_count']+latency['platform_250_500ms_count']
        latency['platform_1_5s_count']=latency['platform_1_3s_count']+latency['platform_3_5s_count']
        latency['delivery_5_30s_count']=latency['delivery_5_10s_count']+latency['delivery_10_15s_count']+latency['delivery_15_30s_count']
        latency['delivery_30s_3m_count']=latency['delivery_30_60s_count']+latency['delivery_60_120s_count']+latency['delivery_120_180s_count']
        latency['delivery_3m_6h_count']=latency['delivery_3_10m_count']+latency['delivery_10_30m_count']+latency['delivery_30_360m_count']
        latency['delivery_over6h_count']=latency['delivery_6_24h_count']+latency['delivery_over24h_count']
        latency['platformlatency_average'] = (latency['platformlatencysum']/latency['platformlatencycount'])
        latency['deliverylatency_average'] = (latency['deliverylatencysum']/latency['deliverylatencycount'])
        #latency = latency.groupby([pd.Grouper(key='requesttime',freq='1H'),'customeraccount','destinationid','supplierid','productid']).agg({'e2elatencycount':'sum','e2elatencysum':'sum','acklatencycount':'sum','acklatencysum':'sum','platform_0_250ms_count':'sum','platform_250_500ms_count':'sum','platform_0_500ms_count':'sum','platform_500_1000ms_count':'sum','platform_1_3s_count':'sum','platform_3_5s_count':'sum','platform_1_5s_count':'sum','platform_5_10s_count':'sum','platform_over10s_count':'sum','platformlatencycount':'sum','platformlatencysum':'sum','platformlatency_average':'sum','delivery_0_5s_count':'sum','delivery_5_10s_count':'sum','delivery_10_15s_count':'sum','delivery_15_30s_count':'sum','delivery_5_30s_count':'sum','delivery_30_60s_count':'sum','delivery_60_120s_count':'sum','delivery_120_180s_count':'sum','delivery_30s_3m_count':'sum','delivery_3_10m_count':'sum','delivery_10_30m_count':'sum','delivery_30_360m_count':'sum','delivery_3m_6h_count':'sum','delivery_6_24h_count':'sum','delivery_over24h_count':'sum','delivery_over6h_count':'sum','deliverylatencycount':'sum','deliverylatencysum':'sum','deliverylatency_average':'sum'})
        latency = latency.groupby([pd.Grouper(key='requesttime',freq='1H'),'customeraccount','destinationid','supplierid','productid']).agg({'e2elatencycount':'sum','e2elatencysum':'sum','acklatencycount':'sum','acklatencysum':'sum','platform_0_250ms_count':'sum','platform_250_500ms_count':'sum','platform_0_500ms_count':'sum','platform_500_1000ms_count':'sum','platform_1_3s_count':'sum','platform_3_5s_count':'sum','platform_1_5s_count':'sum','platform_5_10s_count':'sum','platform_over10s_count':'sum','platformlatencycount':'sum','platformlatencysum':'sum','platformlatency_average':'sum','delivery_0_5s_count':'sum','delivery_5_10s_count':'sum','delivery_10_15s_count':'sum','delivery_15_30s_count':'sum','delivery_5_30s_count':'sum','delivery_30_60s_count':'sum','delivery_60_120s_count':'sum','delivery_120_180s_count':'sum','delivery_30s_3m_count':'sum','delivery_3_10m_count':'sum','delivery_10_30m_count':'sum','delivery_30_360m_count':'sum','delivery_3m_6h_count':'sum','delivery_6_24h_count':'sum','delivery_over24h_count':'sum','delivery_over6h_count':'sum','deliverylatencycount':'sum','deliverylatencysum':'sum','deliverylatency_average':'sum'})
        latency = latency[['e2elatencycount','e2elatencysum','acklatencycount','acklatencysum','platform_0_500ms_count','platform_500_1000ms_count','platform_1_5s_count','platform_5_10s_count','platform_over10s_count','platformlatencycount','platformlatencysum','platformlatency_average','delivery_0_5s_count','delivery_5_30s_count','delivery_30s_3m_count','delivery_3m_6h_count','delivery_over6h_count','deliverylatencycount','deliverylatencysum','deliverylatency_average']]
        #print(latency.groupby('supplierid').agg({'platformlatencysum' :'sum', 'platformlatencycount':'sum'}))
        # latency.drop_duplicates(inplace=True)
        latency = latency.reset_index(level=[0,1,2,3,4])
        #print(latency.groupby('supplierid').agg({'platformlatencysum' :'sum', 'platformlatencycount':'sum'}
        return latency
    def delivery(self):
        delivery=pd.read_csv(self.delivery_data_path)
        vpc = self.destination_of_country()
        delivery['requesttime'] = pd.to_datetime(delivery['requesttime'],dayfirst=True)
        delivery = delivery.sort_values(by=['requesttime']).reset_index(drop=True)
        delivery = delivery.loc[(delivery['customeraccount']==self.customeraccount)& (delivery['destinationid'].isin(vpc))].reset_index(drop=True)
        delivery['requesttime'] = pd.to_datetime(delivery['requesttime'],dayfirst=True)
        delivery['requesttime'] = delivery['requesttime'].dt.strftime('%d-%m-%Y %H:%M')
        delivery['requesttime'] = pd.to_datetime(delivery['requesttime'], format='%d-%m-%Y %H:%M')
        delivery = delivery[['requesttime','customeraccount','supplierid','destinationid','productid','smsattemptsuccesscount', 'smsattemptfailedcount','smssubmitsuccesscount','smssubmitfailedcount','smsdeliversuccesscount', 'smsdeliverfailedcount','submitsuccesscount','submitfailedcount','attemptsuccesscount','attemptfailedcount','deliversuccesscount','deliverfailedcount']]
        delivery = delivery.groupby([pd.Grouper(key='requesttime', freq='1H'),'customeraccount','destinationid','supplierid','productid']).agg({'smsattemptsuccesscount':'sum','smsattemptfailedcount':'sum','smssubmitsuccesscount':'sum','smssubmitfailedcount':'sum','smsdeliversuccesscount':'sum','smsdeliverfailedcount':'sum','submitsuccesscount':'sum','submitfailedcount':'sum','attemptsuccesscount':'sum','attemptfailedcount':'sum','attemptfailedcount':'sum','deliversuccesscount':'sum','deliverfailedcount':'sum'})
        delivery['volume'] = delivery['submitsuccesscount']+delivery['submitfailedcount']
        delivery = delivery.reset_index(level=[0,1,2,3,4])
        return delivery
    def merging(self):
        so2=self.mnp_cost()
        latency=self.latency()
        delivery=self.delivery()
        y = delivery.merge(latency, on=["requesttime","customeraccount","destinationid", "supplierid","productid"], how='outer', indicator=True)
        y.drop_duplicates(inplace=True)
        y.drop('_merge',axis=1,inplace=True)
        a = y.merge(so2, left_on=["destinationid", "supplierid","productid"],right_on=["DEST_VPC","SUPPLIER_ACC_ID","PROD_ID"], how='outer',indicator=True)
        a.drop_duplicates(inplace=True)
        a.drop(["DEST_VPC","SUPPLIER_ACC_ID","PROD_ID"],axis=1,inplace=True)
        a.drop('_merge',axis=1,inplace=True)
        return a
    def adding_delivery_ratio(self):
        a=self.merging()
        for i in range(len(a)):
            if (a.loc[i,"CONNECTIVITY"]=='SMPP'):
                a['delivery_ratio']=(a['deliversuccesscount']/a['volume'])
            elif (a.loc[i,'CONNECTIVITY']=='SS7'):
                a['delivery_ratio']=a['volume']/a['attemptsuccesscount']
        a=a.dropna()
        return a
    def merging_lookups(self):
        a=self.adding_delivery_ratio()
        rg = pd.read_excel(self.Country_Region_Mapping_path)
        rg['COUNTRY'] = rg['COUNTRY'].str.upper()
        rg['REGION'] = rg['REGION'].str.upper()
        rg.loc[rg['COUNTRY']=='UNITED KINGDOM','COUNTRY']="UK"
        cd = a.merge(rg, left_on=["COUNTRY_NM"],right_on=["COUNTRY"], how='left', indicator=True)
        cd.drop_duplicates(inplace=True)
        cd.drop(["COUNTRY"],axis=1,inplace=True)
        cd.drop('_merge',axis=1,inplace=True)
        cust = pd.read_csv(self.customer_lookup_path)
        cust = cust[['customer_acc_id','customer_acc_nm']]
        cust.rename(columns = {'customer_acc_nm':'customer_acc_name'}, inplace = True)
        cust['customer_acc_name'] = cust['customer_acc_name'].str.upper()
        dc = cd.merge(cust, left_on=["customeraccount"],right_on=["customer_acc_id"], how='left', indicator=True)
        dc.drop_duplicates(inplace=True)
        dc.drop(["customeraccount"],axis=1,inplace=True)
        dc.drop('_merge',axis=1,inplace=True)
        clientPrice=pd.read_csv(self.customer_pricelist_path)
        clientPrice = clientPrice[['Country','Network','New Price']]
        clientPrice.rename(columns = {'New Price':'Customer Price'}, inplace = True)
        ef = dc.merge(clientPrice, left_on=["COUNTRY_NM","NETWORK_NM"],right_on=["Country","Network"], how='left', indicator=True)
        ef.drop_duplicates(inplace=True)
        ef.drop(["Country",'Network'],axis=1,inplace=True)
        ef.drop('_merge',axis=1,inplace=True)
        return ef
    def additional_columns(self):
        ef=self.merging_lookups()
        ef.columns=ef.columns.str.upper()
        ef['PROD_COVERAGE']="Y"
        ef.rename(columns = {'COUNTRY_NM':'COUNTRY_NAME'}, inplace = True)
        ef.rename(columns = {'NETWORK_NM':'NETWORK_NAME'}, inplace = True)
        ef.rename(columns = {'PROD_NM':'PRODUCT_NAME'}, inplace = True)
        ef.rename(columns = {'SUPPLIER_ACC_NM':'SUPPLIER_ACC_NAME'}, inplace = True)
        ef["DESTINATION"] = ef["NETWORK_NAME"].astype(str) +" ["+ ef['DESTINATIONID']+"]"
        ef["CUSTOMER"] = ef["CUSTOMER_ACC_NAME"].astype(str) +" ["+ ef['CUSTOMER_ACC_ID'] +"]"
        ef["SUPPLIER"] = ef["SUPPLIER_ACC_NAME"].astype(str) +" ["+ ef['SUPPLIERID'] +"]"
        ef["PRODUCT"] = ef["PRODUCT_NAME"].astype(str) +" ["+ ef['PRODUCTID']+"]"
        datamodel=ef[['REQUESTTIME','REGION','COUNTRY_NAME','DESTINATION','DESTINATIONID','NETWORK_NAME','CUSTOMER','CUSTOMER_ACC_ID', 'CUSTOMER_ACC_NAME','SUPPLIER', 'SUPPLIERID', 'SUPPLIER_ACC_NAME','PRODUCT', 'PRODUCTID', 'PRODUCT_NAME','SMSATTEMPTSUCCESSCOUNT', 'SMSATTEMPTFAILEDCOUNT', 'SMSSUBMITSUCCESSCOUNT', 'SMSSUBMITFAILEDCOUNT', 'SMSDELIVERSUCCESSCOUNT', 'SMSDELIVERFAILEDCOUNT', 'SUBMITSUCCESSCOUNT', 'SUBMITFAILEDCOUNT', 'VOLUME', 'ATTEMPTSUCCESSCOUNT', 'ATTEMPTFAILEDCOUNT', 'DELIVERSUCCESSCOUNT', 'DELIVERFAILEDCOUNT', 'DELIVERY_RATIO', 'E2ELATENCYCOUNT', 'E2ELATENCYSUM', 'ACKLATENCYCOUNT', 'ACKLATENCYSUM', 'PLATFORM_0_500MS_COUNT', 'PLATFORM_500_1000MS_COUNT', 'PLATFORM_1_5S_COUNT', 'PLATFORM_5_10S_COUNT', 'PLATFORM_OVER10S_COUNT', 'PLATFORMLATENCYCOUNT', 'PLATFORMLATENCYSUM', 'PLATFORMLATENCY_AVERAGE', 'DELIVERY_0_5S_COUNT', 'DELIVERY_5_30S_COUNT', 'DELIVERY_30S_3M_COUNT', 'DELIVERY_3M_6H_COUNT', 'DELIVERY_OVER6H_COUNT', 'DELIVERYLATENCYCOUNT', 'DELIVERYLATENCYSUM', 'DELIVERYLATENCY_AVERAGE','CUSTOMER PRICE','PROD_COVERAGE', 'CONNECTIVITY', 'MCC', 'MNC', 'SUPP_DLR_SUPP', 'CHARGING_MODEL', 'TPS', 'MT_COST', 'MNP COST', 'COST_PER_SMS', 'COVERAGE_FL', 'CONCAT_SUPPORT', 'ROUTE_TYPE', 'MNP_SUPPORT', 'DATACODE_SUPPORT', 'DR_HANDSET', 'DR_SMSC']]
        return datamodel
    def downloading(self):
        final=self.additional_columns()
        download_path='/Users/kaushiki./Downloads/FINAL_DATAMODEL_'+str(self.product_id)+'_'+str(self.COUNTRY_NAME)+'.csv'
        final=final.to_csv(download_path, sep=',', encoding='utf-8')
        return final


In [216]:
datamodel=Datamodel('/Users/kaushiki./Downloads/Supplier Data Updated/PRD031_BRAZIL_SUPPLIER_DATA.csv','/Users/kaushiki./Downloads/mmx3_prod_route_matrix 2.csv','/Users/kaushiki./Downloads/MNP-HLR Provider cost v0.xlsx','Mx-OTT','PRD031','BRAZIL','/Users/kaushiki./Downloads/destination_lookup.csv','29506-2','/Users/kaushiki./Downloads/Axiom MVP data 11-10-2022/LatencyStatistic_Data/PRD031_Brazil_Latency_JUL-SEP.csv','/Users/kaushiki./Downloads/Axiom MVP data 11-10-2022/DeliveryStatistic_Data/PRD031_Brazil_Delivery-JUL-SEP.csv','/Users/kaushiki./Downloads/Country Region Mapping.xlsx','/Users/kaushiki./Downloads/LookupDataJune27/customer_lookup.csv','/Users/kaushiki./Downloads/Customer Pricelist/META PLATFORMS INC.-FACEBOOK-SMPP-OTT_11-10-2022.csv')

In [205]:
datamodel.destination_of_country()

['6-238-0',
 '1-173-0',
 '4-124-0',
 '2-024-0',
 '2-152-0',
 '2-029-0',
 '3-215-0',
 '2-101-0',
 '3-094-0',
 '2-228-0',
 '5-055-0',
 '6-084-0',
 '6-242-0',
 '7-199-0',
 '2-246-2',
 '2-246-3']

In [192]:
datamodel.supplier()

,PROD_ID,SUPPLIER_ACC_ID,SUPP_ACC_INST,SUPP_SMPP_MAX_CONN,COUNTRY_NM,PROD_NM,SUPPLIER_ACC_NM,NETWORK_NM,CONNECTIVITY,DEST_VPC,...,CHARGING_MODEL,SUPP_TRFF_THROTT,MT_COST,COVERAGE_FL,CONCAT_SUPPORT,ROUTE_TYPE,MNP_SUPPORT,DATACODE_SUPPORT,DR_HANDSET,DR_SMSC
0,PRD031,27703-1,MTTSMSHUB3,10,BRAZIL,Mx-OTT,LLEIDA-MT-SMPP,VIVO,SMPP,3-094-0,...,1,400,0.006,Y,N,None,N,Text,N,Y
1,PRD031,28242-1,MTTSMSHUB3,1,BRAZIL,Mx-OTT,BFTTELECOM-MT-SMPP,GLOBALSTAR BRAZIL,SMPP,1-173-0,...,2,100,0.00485,Y,Y,Partner Direct,N,Text,N,Y
2,PRD031,28242-1,MTTSMSHUB3,1,BRAZIL,Mx-OTT,BFTTELECOM-MT-SMPP,TELEMIG CELULAR VIVO MG BRAZIL,SMPP,2-228-0,...,2,100,0.00485,Y,Y,Partner Direct,N,Text,N,Y
3,PRD031,29042-2,MTTSMSHUB3,1,BRAZIL,Mx-OTT,TIWS-MT-SMPP,TELEMIG CELULAR VIVO MG BRAZIL,SMPP,2-228-0,...,1,100,0.01,Y,N,Direct,N,Text,N,N
4,PRD031,29042-2,MTTSMSHUB3,1,BRAZIL,Mx-OTT,TIWS-MT-SMPP,VIVO,SMPP,3-094-0,...,1,100,0.01,Y,N,Direct,N,Text,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207,PRD031,60465-1,MTTSMSHUB3,5,BRAZIL,Mx-OTT,GLOBALVOICE-MT-SMPP,GLOBALSTAR BRAZIL,SMPP,1-173-0,...,1,200,0.003,Y,N,Partner Direct,N,Text,N,Y
1208,PRD031,6067-1,MTTSMSHUB3,20,BRAZIL,Mx-OTT,CAPITAL-MT-SMPP,SURF 4G,SMPP,6-242-0,...,1,130,0.0007,Y,N,Partner Direct,N,Text,N,N
1209,PRD031,6067-1,TTTSMSHUB3,20,BRAZIL,Mx-OTT,CAPITAL-MT-SMPP,TELEMIG CELULAR VIVO MG BRAZIL,SMPP,2-228-0,...,1,130,0.0007,Y,N,Partner Direct,N,Text,N,N
1210,PRD031,6067-1,TTTSMSHUB3,20,BRAZIL,Mx-OTT,CAPITAL-MT-SMPP,CLARO BRAZIL,SMPP,2-101-0,...,1,130,0.0007,Y,N,Partner Direct,N,Text,N,N


In [193]:
datamodel.adding_tps()

/var/folders/jd/h5cmmg656z39my6vfpb_tcy40000gn/T/ipykernel_1076/2142189885.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  supp1['TPS']= (supp1['SUPP_TRFF_THROTT'].astype(int)*supp1['SUPP_SMPP_MAX_CONN'].astype(int))
/var/folders/jd/h5cmmg656z39my6vfpb_tcy40000gn/T/ipykernel_1076/2142189885.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  so.drop_duplicates(inplace=True)


,PROD_ID,SUPPLIER_ACC_ID,COUNTRY_NM,PROD_NM,SUPPLIER_ACC_NM,NETWORK_NM,CONNECTIVITY,DEST_VPC,MCC,MNC,...,CHARGING_MODEL,MT_COST,COVERAGE_FL,CONCAT_SUPPORT,ROUTE_TYPE,MNP_SUPPORT,DATACODE_SUPPORT,DR_HANDSET,DR_SMSC,TPS
0,PRD031,27703-1,BRAZIL,Mx-OTT,LLEIDA-MT-SMPP,VIVO,SMPP,3-094-0,724,"01, 06, 10, 11, 19",...,1,0.006,Y,N,None,N,Text,N,Y,4400
1,PRD031,28242-1,BRAZIL,Mx-OTT,BFTTELECOM-MT-SMPP,GLOBALSTAR BRAZIL,SMPP,1-173-0,724,88,...,2,0.00485,Y,Y,Partner Direct,N,Text,N,Y,200
2,PRD031,28242-1,BRAZIL,Mx-OTT,BFTTELECOM-MT-SMPP,TELEMIG CELULAR VIVO MG BRAZIL,SMPP,2-228-0,724,23,...,2,0.00485,Y,Y,Partner Direct,N,Text,N,Y,200
3,PRD031,29042-2,BRAZIL,Mx-OTT,TIWS-MT-SMPP,TELEMIG CELULAR VIVO MG BRAZIL,SMPP,2-228-0,724,23,...,1,0.01,Y,N,Direct,N,Text,N,N,200
4,PRD031,29042-2,BRAZIL,Mx-OTT,TIWS-MT-SMPP,VIVO,SMPP,3-094-0,724,"01, 06, 10, 11, 19",...,1,0.01,Y,N,Direct,N,Text,N,N,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,PRD031,40654-1,BRAZIL,Mx-OTT,BEEPSEND-MT-SMPP-DIRECT,VIVO,SMPP,3-094-0,724,"01, 06, 10, 11, 19",...,1,0.01525,Y,N,Partner Direct,N,Text,N,N,200
1106,PRD031,59275-2,BRAZIL,Mx-OTT,CHINATELECOM-MT-SMPP-TRANS,TELEMIG CELULAR VIVO MG BRAZIL,SMPP,2-228-0,724,23,...,1,0.006,Y,Y,Partner Direct,N,Text,N,Y,120
1111,PRD031,60465-1,BRAZIL,Mx-OTT,GLOBALVOICE-MT-SMPP,TELEMIG CELULAR VIVO MG BRAZIL,SMPP,2-228-0,724,23,...,1,0.003,Y,N,Partner Direct,N,Text,N,Y,2000
1127,PRD031,35336-2,BRAZIL,Mx-OTT,MRMESSAGING-MT-SMPP-TRANS,CINCO,SMPP,7-199-0,724,14,...,1,0.0049,Y,N,Partner Direct,N,Text,N,N,800


In [194]:
datamodel.mnp_cost()

/var/folders/jd/h5cmmg656z39my6vfpb_tcy40000gn/T/ipykernel_1076/2142189885.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  supp1['TPS']= (supp1['SUPP_TRFF_THROTT'].astype(int)*supp1['SUPP_SMPP_MAX_CONN'].astype(int))
/var/folders/jd/h5cmmg656z39my6vfpb_tcy40000gn/T/ipykernel_1076/2142189885.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  so.drop_duplicates(inplace=True)


,PROD_ID,SUPPLIER_ACC_ID,COUNTRY_NM,PROD_NM,SUPPLIER_ACC_NM,NETWORK_NM,CONNECTIVITY,DEST_VPC,MCC,MNC,...,COVERAGE_FL,CONCAT_SUPPORT,ROUTE_TYPE,MNP_SUPPORT,DATACODE_SUPPORT,DR_HANDSET,DR_SMSC,TPS,MNP COST,COST_PER_SMS
0,PRD031,27703-1,BRAZIL,Mx-OTT,LLEIDA-MT-SMPP,VIVO,SMPP,3-094-0,724,"01, 06, 10, 11, 19",...,Y,N,None,N,Text,N,Y,4400,0,0.00600
1,PRD031,28242-1,BRAZIL,Mx-OTT,BFTTELECOM-MT-SMPP,GLOBALSTAR BRAZIL,SMPP,1-173-0,724,88,...,Y,Y,Partner Direct,N,Text,N,Y,200,0,0.00485
2,PRD031,28242-1,BRAZIL,Mx-OTT,BFTTELECOM-MT-SMPP,TELEMIG CELULAR VIVO MG BRAZIL,SMPP,2-228-0,724,23,...,Y,Y,Partner Direct,N,Text,N,Y,200,0,0.00485
3,PRD031,29042-2,BRAZIL,Mx-OTT,TIWS-MT-SMPP,TELEMIG CELULAR VIVO MG BRAZIL,SMPP,2-228-0,724,23,...,Y,N,Direct,N,Text,N,N,200,0,0.01000
4,PRD031,29042-2,BRAZIL,Mx-OTT,TIWS-MT-SMPP,VIVO,SMPP,3-094-0,724,"01, 06, 10, 11, 19",...,Y,N,Direct,N,Text,N,N,200,0,0.01000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,PRD031,40654-1,BRAZIL,Mx-OTT,BEEPSEND-MT-SMPP-DIRECT,VIVO,SMPP,3-094-0,724,"01, 06, 10, 11, 19",...,Y,N,Partner Direct,N,Text,N,N,200,0,0.01525
1106,PRD031,59275-2,BRAZIL,Mx-OTT,CHINATELECOM-MT-SMPP-TRANS,TELEMIG CELULAR VIVO MG BRAZIL,SMPP,2-228-0,724,23,...,Y,Y,Partner Direct,N,Text,N,Y,120,0,0.00600
1111,PRD031,60465-1,BRAZIL,Mx-OTT,GLOBALVOICE-MT-SMPP,TELEMIG CELULAR VIVO MG BRAZIL,SMPP,2-228-0,724,23,...,Y,N,Partner Direct,N,Text,N,Y,2000,0,0.00300
1127,PRD031,35336-2,BRAZIL,Mx-OTT,MRMESSAGING-MT-SMPP-TRANS,CINCO,SMPP,7-199-0,724,14,...,Y,N,Partner Direct,N,Text,N,N,800,0,0.00490


In [196]:
datamodel.latency()

,requesttime,customeraccount,destinationid,supplierid,productid,e2elatencycount,e2elatencysum,acklatencycount,acklatencysum,platform_0_500ms_count,...,platformlatencysum,platformlatency_average,delivery_0_5s_count,delivery_5_30s_count,delivery_30s_3m_count,delivery_3m_6h_count,delivery_over6h_count,deliverylatencycount,deliverylatencysum,deliverylatency_average
0,2022-07-01 02:00:00,29506-2,2-029-0,19598-8,PRD031,1,7.470,1,1,1,...,12,12.000000,1,0,0,0,0,1,0.066,0.066000
1,2022-07-01 04:00:00,29506-2,2-101-0,19598-8,PRD031,1,2.632,1,1,1,...,8,8.000000,1,0,0,0,0,1,0.044,0.044000
2,2022-07-01 05:00:00,29506-2,2-029-0,19598-8,PRD031,1,4.557,1,1,1,...,5,5.000000,1,0,0,0,0,1,0.049,0.049000
3,2022-07-01 05:00:00,29506-2,2-101-0,19598-8,PRD031,1,13.297,1,1,1,...,15,15.000000,1,0,0,0,0,1,0.040,0.040000
4,2022-07-01 06:00:00,29506-2,2-029-0,19598-8,PRD031,3,20.869,4,4,4,...,115,43.666667,4,0,0,0,0,4,0.170,0.080667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4513,2022-09-30 23:00:00,29506-2,2-029-0,19598-8,PRD031,4,55.953,4,4,4,...,55,55.000000,4,0,0,0,0,4,0.184,0.184000
4514,2022-09-30 23:00:00,29506-2,2-101-0,19598-8,PRD031,13,221.075,18,18,18,...,183,63.416667,18,0,0,0,0,18,0.874,0.342217
4515,2022-09-30 23:00:00,29506-2,2-228-0,19598-8,PRD031,7,103.001,8,8,8,...,72,48.750000,8,0,0,0,0,8,0.495,0.351750
4516,2022-09-30 23:00:00,29506-2,2-246-2,35192-1,PRD031,0,0.000,1,1,1,...,7,7.000000,0,0,0,0,0,0,0.000,0.000000


In [197]:
datamodel.delivery()

,requesttime,customeraccount,destinationid,supplierid,productid,smsattemptsuccesscount,smsattemptfailedcount,smssubmitsuccesscount,smssubmitfailedcount,smsdeliversuccesscount,smsdeliverfailedcount,submitsuccesscount,submitfailedcount,attemptsuccesscount,attemptfailedcount,deliversuccesscount,deliverfailedcount,volume
0,2022-07-01 02:00:00,29506-2,2-029-0,19598-8,PRD031,1,0,1,0,1,0,1,0,1,0,1,0,1
1,2022-07-01 04:00:00,29506-2,2-101-0,19598-8,PRD031,1,0,1,0,1,0,1,0,1,0,1,0,1
2,2022-07-01 05:00:00,29506-2,2-029-0,19598-8,PRD031,1,0,1,0,1,0,1,0,1,0,1,0,1
3,2022-07-01 05:00:00,29506-2,2-101-0,19598-8,PRD031,1,0,1,0,1,0,1,0,1,0,1,0,1
4,2022-07-01 06:00:00,29506-2,2-029-0,19598-8,PRD031,4,0,4,0,3,0,4,0,4,0,3,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4513,2022-09-30 23:00:00,29506-2,2-029-0,19598-8,PRD031,4,0,4,0,4,0,4,0,4,0,4,0,4
4514,2022-09-30 23:00:00,29506-2,2-101-0,19598-8,PRD031,18,0,18,0,13,4,18,0,18,0,13,4,18
4515,2022-09-30 23:00:00,29506-2,2-228-0,19598-8,PRD031,8,0,8,0,7,0,8,0,8,0,7,0,8
4516,2022-09-30 23:00:00,29506-2,2-246-2,35192-1,PRD031,1,0,0,0,0,0,0,0,1,0,0,0,0


In [198]:
datamodel.merging()

/var/folders/jd/h5cmmg656z39my6vfpb_tcy40000gn/T/ipykernel_1076/2142189885.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  supp1['TPS']= (supp1['SUPP_TRFF_THROTT'].astype(int)*supp1['SUPP_SMPP_MAX_CONN'].astype(int))
/var/folders/jd/h5cmmg656z39my6vfpb_tcy40000gn/T/ipykernel_1076/2142189885.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  so.drop_duplicates(inplace=True)


,requesttime,customeraccount,destinationid,supplierid,productid,smsattemptsuccesscount,smsattemptfailedcount,smssubmitsuccesscount,smssubmitfailedcount,smsdeliversuccesscount,...,COVERAGE_FL,CONCAT_SUPPORT,ROUTE_TYPE,MNP_SUPPORT,DATACODE_SUPPORT,DR_HANDSET,DR_SMSC,TPS,MNP COST,COST_PER_SMS
0,2022-07-01 02:00:00,29506-2,2-029-0,19598-8,PRD031,1.0,0.0,1.0,0.0,1.0,...,Y,Y,Partner Direct,N,Text,N,Y,200.0,0.0,0.00600
1,2022-07-01 05:00:00,29506-2,2-029-0,19598-8,PRD031,1.0,0.0,1.0,0.0,1.0,...,Y,Y,Partner Direct,N,Text,N,Y,200.0,0.0,0.00600
2,2022-07-01 06:00:00,29506-2,2-029-0,19598-8,PRD031,4.0,0.0,4.0,0.0,3.0,...,Y,Y,Partner Direct,N,Text,N,Y,200.0,0.0,0.00600
3,2022-07-01 07:00:00,29506-2,2-029-0,19598-8,PRD031,9.0,0.0,9.0,0.0,8.0,...,Y,Y,Partner Direct,N,Text,N,Y,200.0,0.0,0.00600
4,2022-07-01 08:00:00,29506-2,2-029-0,19598-8,PRD031,1.0,0.0,1.0,0.0,2.0,...,Y,Y,Partner Direct,N,Text,N,Y,200.0,0.0,0.00600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5105,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Y,N,Partner Direct,N,Text,N,N,200.0,0.0,0.01525
5106,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Y,Y,Partner Direct,N,Text,N,Y,120.0,0.0,0.00600
5107,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Y,N,Partner Direct,N,Text,N,Y,2000.0,0.0,0.00300
5108,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Y,N,Partner Direct,N,Text,N,N,800.0,0.0,0.00490


In [199]:
datamodel.adding_delivery_ratio()

/var/folders/jd/h5cmmg656z39my6vfpb_tcy40000gn/T/ipykernel_1076/2142189885.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  supp1['TPS']= (supp1['SUPP_TRFF_THROTT'].astype(int)*supp1['SUPP_SMPP_MAX_CONN'].astype(int))
/var/folders/jd/h5cmmg656z39my6vfpb_tcy40000gn/T/ipykernel_1076/2142189885.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  so.drop_duplicates(inplace=True)


,requesttime,customeraccount,destinationid,supplierid,productid,smsattemptsuccesscount,smsattemptfailedcount,smssubmitsuccesscount,smssubmitfailedcount,smsdeliversuccesscount,...,CONCAT_SUPPORT,ROUTE_TYPE,MNP_SUPPORT,DATACODE_SUPPORT,DR_HANDSET,DR_SMSC,TPS,MNP COST,COST_PER_SMS,delivery_ratio
0,2022-07-01 02:00:00,29506-2,2-029-0,19598-8,PRD031,1.0,0.0,1.0,0.0,1.0,...,Y,Partner Direct,N,Text,N,Y,200.0,0.0,0.00600,1.000000
1,2022-07-01 05:00:00,29506-2,2-029-0,19598-8,PRD031,1.0,0.0,1.0,0.0,1.0,...,Y,Partner Direct,N,Text,N,Y,200.0,0.0,0.00600,1.000000
2,2022-07-01 06:00:00,29506-2,2-029-0,19598-8,PRD031,4.0,0.0,4.0,0.0,3.0,...,Y,Partner Direct,N,Text,N,Y,200.0,0.0,0.00600,0.750000
3,2022-07-01 07:00:00,29506-2,2-029-0,19598-8,PRD031,9.0,0.0,9.0,0.0,8.0,...,Y,Partner Direct,N,Text,N,Y,200.0,0.0,0.00600,0.888889
4,2022-07-01 08:00:00,29506-2,2-029-0,19598-8,PRD031,1.0,0.0,1.0,0.0,2.0,...,Y,Partner Direct,N,Text,N,Y,200.0,0.0,0.00600,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4513,2022-09-08 15:00:00,29506-2,5-055-0,29923-4,PRD031,2.0,0.0,2.0,0.0,2.0,...,N,None,N,Text,N,N,400.0,0.0,0.00475,1.000000
4514,2022-09-08 16:00:00,29506-2,5-055-0,29923-4,PRD031,2.0,0.0,2.0,0.0,2.0,...,N,None,N,Text,N,N,400.0,0.0,0.00475,1.000000
4515,2022-09-08 17:00:00,29506-2,5-055-0,29923-4,PRD031,3.0,0.0,3.0,0.0,3.0,...,N,None,N,Text,N,N,400.0,0.0,0.00475,1.000000
4516,2022-09-14 04:00:00,29506-2,5-055-0,29923-4,PRD031,1.0,0.0,1.0,0.0,1.0,...,N,None,N,Text,N,N,400.0,0.0,0.00475,1.000000


In [200]:
datamodel.merging_lookups()

/var/folders/jd/h5cmmg656z39my6vfpb_tcy40000gn/T/ipykernel_1076/2142189885.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  supp1['TPS']= (supp1['SUPP_TRFF_THROTT'].astype(int)*supp1['SUPP_SMPP_MAX_CONN'].astype(int))
/var/folders/jd/h5cmmg656z39my6vfpb_tcy40000gn/T/ipykernel_1076/2142189885.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  so.drop_duplicates(inplace=True)


,requesttime,destinationid,supplierid,productid,smsattemptsuccesscount,smsattemptfailedcount,smssubmitsuccesscount,smssubmitfailedcount,smsdeliversuccesscount,smsdeliverfailedcount,...,DR_HANDSET,DR_SMSC,TPS,MNP COST,COST_PER_SMS,delivery_ratio,REGION,customer_acc_id,customer_acc_name,Customer Price
0,2022-07-01 02:00:00,2-029-0,19598-8,PRD031,1.0,0.0,1.0,0.0,1.0,0.0,...,N,Y,200.0,0.0,0.00600,1.000000,AMERICA,29506-2,FACEBOOK-SMPP-OTT,0.0090
1,2022-07-01 05:00:00,2-029-0,19598-8,PRD031,1.0,0.0,1.0,0.0,1.0,0.0,...,N,Y,200.0,0.0,0.00600,1.000000,AMERICA,29506-2,FACEBOOK-SMPP-OTT,0.0090
2,2022-07-01 06:00:00,2-029-0,19598-8,PRD031,4.0,0.0,4.0,0.0,3.0,0.0,...,N,Y,200.0,0.0,0.00600,0.750000,AMERICA,29506-2,FACEBOOK-SMPP-OTT,0.0090
3,2022-07-01 07:00:00,2-029-0,19598-8,PRD031,9.0,0.0,9.0,0.0,8.0,0.0,...,N,Y,200.0,0.0,0.00600,0.888889,AMERICA,29506-2,FACEBOOK-SMPP-OTT,0.0090
4,2022-07-01 08:00:00,2-029-0,19598-8,PRD031,1.0,0.0,1.0,0.0,2.0,0.0,...,N,Y,200.0,0.0,0.00600,2.000000,AMERICA,29506-2,FACEBOOK-SMPP-OTT,0.0090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3419,2022-09-08 15:00:00,5-055-0,29923-4,PRD031,2.0,0.0,2.0,0.0,2.0,0.0,...,N,N,400.0,0.0,0.00475,1.000000,AMERICA,29506-2,FACEBOOK-SMPP-OTT,0.0062
3420,2022-09-08 16:00:00,5-055-0,29923-4,PRD031,2.0,0.0,2.0,0.0,2.0,0.0,...,N,N,400.0,0.0,0.00475,1.000000,AMERICA,29506-2,FACEBOOK-SMPP-OTT,0.0062
3421,2022-09-08 17:00:00,5-055-0,29923-4,PRD031,3.0,0.0,3.0,0.0,3.0,0.0,...,N,N,400.0,0.0,0.00475,1.000000,AMERICA,29506-2,FACEBOOK-SMPP-OTT,0.0062
3422,2022-09-14 04:00:00,5-055-0,29923-4,PRD031,1.0,0.0,1.0,0.0,1.0,0.0,...,N,N,400.0,0.0,0.00475,1.000000,AMERICA,29506-2,FACEBOOK-SMPP-OTT,0.0062


In [201]:
datamodel.additional_columns()

/var/folders/jd/h5cmmg656z39my6vfpb_tcy40000gn/T/ipykernel_1076/2142189885.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  supp1['TPS']= (supp1['SUPP_TRFF_THROTT'].astype(int)*supp1['SUPP_SMPP_MAX_CONN'].astype(int))
/var/folders/jd/h5cmmg656z39my6vfpb_tcy40000gn/T/ipykernel_1076/2142189885.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  so.drop_duplicates(inplace=True)


,REQUESTTIME,REGION,COUNTRY_NAME,DESTINATION,DESTINATIONID,NETWORK_NAME,CUSTOMER,CUSTOMER_ACC_ID,CUSTOMER_ACC_NAME,SUPPLIER,...,MT_COST,MNP COST,COST_PER_SMS,COVERAGE_FL,CONCAT_SUPPORT,ROUTE_TYPE,MNP_SUPPORT,DATACODE_SUPPORT,DR_HANDSET,DR_SMSC
0,2022-07-01 02:00:00,AMERICA,BRAZIL,TIM BRASIL [2-029-0],2-029-0,TIM BRASIL,FACEBOOK-SMPP-OTT [29506-2],29506-2,FACEBOOK-SMPP-OTT,IDENTIDAD-US-MT-SMPP-OTP [19598-8],...,0.006,0.0,0.00600,Y,Y,Partner Direct,N,Text,N,Y
1,2022-07-01 05:00:00,AMERICA,BRAZIL,TIM BRASIL [2-029-0],2-029-0,TIM BRASIL,FACEBOOK-SMPP-OTT [29506-2],29506-2,FACEBOOK-SMPP-OTT,IDENTIDAD-US-MT-SMPP-OTP [19598-8],...,0.006,0.0,0.00600,Y,Y,Partner Direct,N,Text,N,Y
2,2022-07-01 06:00:00,AMERICA,BRAZIL,TIM BRASIL [2-029-0],2-029-0,TIM BRASIL,FACEBOOK-SMPP-OTT [29506-2],29506-2,FACEBOOK-SMPP-OTT,IDENTIDAD-US-MT-SMPP-OTP [19598-8],...,0.006,0.0,0.00600,Y,Y,Partner Direct,N,Text,N,Y
3,2022-07-01 07:00:00,AMERICA,BRAZIL,TIM BRASIL [2-029-0],2-029-0,TIM BRASIL,FACEBOOK-SMPP-OTT [29506-2],29506-2,FACEBOOK-SMPP-OTT,IDENTIDAD-US-MT-SMPP-OTP [19598-8],...,0.006,0.0,0.00600,Y,Y,Partner Direct,N,Text,N,Y
4,2022-07-01 08:00:00,AMERICA,BRAZIL,TIM BRASIL [2-029-0],2-029-0,TIM BRASIL,FACEBOOK-SMPP-OTT [29506-2],29506-2,FACEBOOK-SMPP-OTT,IDENTIDAD-US-MT-SMPP-OTP [19598-8],...,0.006,0.0,0.00600,Y,Y,Partner Direct,N,Text,N,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3419,2022-09-08 15:00:00,AMERICA,BRAZIL,NEXTEL TELECOMUNICACOES 3G [5-055-0],5-055-0,NEXTEL TELECOMUNICACOES 3G,FACEBOOK-SMPP-OTT [29506-2],29506-2,FACEBOOK-SMPP-OTT,NEXMO-MT-SMPP-DIRECT [29923-4],...,0.00475,0.0,0.00475,Y,N,None,N,Text,N,N
3420,2022-09-08 16:00:00,AMERICA,BRAZIL,NEXTEL TELECOMUNICACOES 3G [5-055-0],5-055-0,NEXTEL TELECOMUNICACOES 3G,FACEBOOK-SMPP-OTT [29506-2],29506-2,FACEBOOK-SMPP-OTT,NEXMO-MT-SMPP-DIRECT [29923-4],...,0.00475,0.0,0.00475,Y,N,None,N,Text,N,N
3421,2022-09-08 17:00:00,AMERICA,BRAZIL,NEXTEL TELECOMUNICACOES 3G [5-055-0],5-055-0,NEXTEL TELECOMUNICACOES 3G,FACEBOOK-SMPP-OTT [29506-2],29506-2,FACEBOOK-SMPP-OTT,NEXMO-MT-SMPP-DIRECT [29923-4],...,0.00475,0.0,0.00475,Y,N,None,N,Text,N,N
3422,2022-09-14 04:00:00,AMERICA,BRAZIL,NEXTEL TELECOMUNICACOES 3G [5-055-0],5-055-0,NEXTEL TELECOMUNICACOES 3G,FACEBOOK-SMPP-OTT [29506-2],29506-2,FACEBOOK-SMPP-OTT,NEXMO-MT-SMPP-DIRECT [29923-4],...,0.00475,0.0,0.00475,Y,N,None,N,Text,N,N


In [217]:
datamodel.downloading()

/var/folders/jd/h5cmmg656z39my6vfpb_tcy40000gn/T/ipykernel_1076/658500489.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  supp1['TPS']= (supp1['SUPP_TRFF_THROTT'].astype(int)*supp1['SUPP_SMPP_MAX_CONN'].astype(int))
/var/folders/jd/h5cmmg656z39my6vfpb_tcy40000gn/T/ipykernel_1076/658500489.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  so.drop_duplicates(inplace=True)


In [72]:
final.isna().sum()

REQUESTTIME         0
REGION              0
COUNTRY_NAME        0
DESTINATION         0
DESTINATIONID       0
                   ..
ROUTE_TYPE          0
MNP_SUPPORT         0
DATACODE_SUPPORT    0
DR_HANDSET          0
DR_SMSC             0
Length: 67, dtype: int64

In [73]:
final.shape

(3424, 67)

# Merge latency/delivery data into one single file

In [1]:
import glob
import pandas as pd
#this class will help us to make one single file(csv) for latency/delivery for each product-country combination
#since tcl has given data for each month, we need to merge it in one single file
class one_file:
    def __init__(self,path,customeraccount,destination_lookup_path,productid,COUNTRY_NAME):
        self.path = path   #path is the folder where all csv's are kept, no need to add file name, keep all in one folder and give folder path
        self.customeraccount = customeraccount
        self.destination_lookup_path = destination_lookup_path
        self.productid = productid
        self.COUNTRY_NAME = COUNTRY_NAME
    def destination_of_country(self):
        dest_lookup=pd.read_csv(self.destination_lookup_path)
        country_name = self.COUNTRY_NAME
        dest_lookup=dest_lookup.loc[dest_lookup['country_nm']==country_name]
        vpc=dest_lookup['vpc'].to_list()
        return vpc
    def one_latency(self):
        csv_files = glob.glob(self.path + "/*.csv")
        download_path = '/Users/kaushiki./Downloads/'+'Latency'+'_'+str(self.productid)+'_'+str(self.COUNTRY_NAME)+'.csv'
        vpc = self.destination_of_country()
        appended_data = []
        for infile in csv_files:
            data = pd.read_csv(infile)
            latency1 = data.sort_values(by=['requesttime']).reset_index(drop=True)
            latency1=latency1.loc[(latency1['productid']==self.productid)]
            sk_latency1 = latency1.loc[(latency1['customeraccount']==self.customeraccount) & (latency1['destinationid'].isin(vpc))].reset_index(drop=True)
            sk_latency1['requesttime'] = pd.to_datetime(sk_latency1['requesttime'])
            sk_latency1['requesttime'] = sk_latency1['requesttime'].dt.strftime('%d-%m-%Y %H:%M')
            sk_latency1['requesttime'] = pd.to_datetime(sk_latency1['requesttime'], format='%d-%m-%Y %H:%M')
            #sk_latency1 = sk_latency1.loc[(sk_latency1['requesttime']>='01/01/2021')].reset_index(drop=True)
            # store DataFrame in list
            appended_data.append(sk_latency1)
        # see pd.concat documentation for more info
        appended_data = pd.concat(appended_data)
        # write DataFrame to an excel sheet 
        appended_data = appended_data.to_csv(download_path, sep=',', encoding='utf-8')
        return appended_data
        

In [2]:
nn=one_file('/Users/kaushiki./Downloads/AGGREGATED_DATA','35308-1','/Users/kaushiki./Downloads/destination_lookup.csv','PRD001','INDIA')

In [3]:
a=nn.one_latency()

In [4]:
a